In [1]:
import torch
from transformers import CamembertForSequenceClassification, CamembertTokenizer, CamembertModel
from torch.utils.data import Dataset

In [2]:
camembert = CamembertForSequenceClassification.from_pretrained('camembert-base',
                                                               num_labels=1)
tokeniser=CamembertTokenizer.from_pretrained('camembert-base')

In [3]:
class FactsOrAnalysisDS_BERT(Dataset):
    """PyTorch Dataset class. Returns input-target tensor pairs"""
    def __init__(self,csv_file,tokeniser,n_read='all'):
        with open(csv_file) as f:
            dataset=[line.split(",") for line in f]
        if n_read=='all':
            self.dataset=[[torch.tensor(tokeniser.encode(datum[0])),
                           int(datum[1][0])] for datum in dataset]
        else:
            self.dataset=[]
            for i in range(n_read):
                self.dataset.append([torch.tensor(tokeniser.encode(dataset[i][0])),
                           int(dataset[i][1][0])])
    
    def __len__(self):
        return (len(self.dataset))
    
    def __getitem__(self,index):
        return self.dataset[index]

In [4]:
data_file="../data/train_data.csv"
with open(data_file) as f:
    dataset=[line.split(",") for line in f]
dataset=[[pair[0].replace("\n",""),pair[1].replace("\n","")] for pair in dataset]

In [5]:
dataset_camembert=FactsOrAnalysisDS_BERT(data_file,tokeniser,n_read=256)

In [37]:
import torch
rnn=torch.nn.GRU(
    input_size=768,
    hidden_size=64,
    num_layers=3,
    batch_first=True,
    bidirectional=True)

linear=torch.nn.Linear(32005,1)

I have to rewrite the `Dataset` class to accomodate camemBERT

In [212]:
# class camembertTest(torch.nn.Module):
#     def __init__(self,cam):
#         super(camembertTest, self).__init__()
#         self.camembert=cam
#         self.sigma=torch.nn.Sigmoid()
        
#     def forward(self,input_tensor):
#         print(self.camembert(input_tensor)[0].shape)
#         print(self.camembert(input_tensor)[0])
#         return self.sigma(self.camembert(input_tensor)[0])

# model=camembertTest(camembert)
model=CamembertModel.from_pretrained('camembert-base')

In [6]:
u=dataset_camembert[255][0].unsqueeze(0)
# u=u.unsqueeze(0)
# u=camembert.encode(u)
# u=camembert.extract_features(u)
# u,v=rnn(u)
# v.shape
# model.eval()


In [10]:
u.is_cuda

False

In [41]:
from transformers import CamembertForSequenceClassification

In [97]:
camembert=CamembertForSequenceClassification.from_pretrained('camembert-base',num_labels=1)

In [53]:
camembert.eval()

CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [80]:

tokeniser = CamembertTokenizer.from_pretrained('camembert-base')

In [106]:
s=" ceci n'est pas une lune"
s=tokeniser.encode(s)
s=torch.tensor(s)
s=s.unsqueeze(0)
s=camembert(s)
s[1]

IndexError: tuple index out of range

In [2]:
from s
data_file = "../data/data.csv"


TypeError: Dataset() takes no arguments